In [3]:
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=2):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def resnet18(pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model

In [ ]:
import torch.utils.data as data
import csv
from PIL import Image as PImage
import torchvision.transforms as transforms
import numpy as np
import torch
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
from torch.autograd import Variable

class CD_Dataset(data.Dataset):
    def __init__(self, src, path_csv, label_csv, transform=None):
        with open(path_csv, 'r') as f:
            path_reader = csv.reader(f)
            p_list = list(path_reader)
           
        with open(label_csv, 'r') as f:
            label_reader = csv.reader(f)
            l_list = list(label_reader)

        self.labels = []
        self.pathes = []

        for i in range(len(p_list)):
            self.pathes.append(*p_list[i])
            self.pathes[i] = src + self.pathes[i]
            #print(self.pathes[i])
        for i in range(len(l_list)):
            self.labels.append(*l_list[i][1])
    
        self.transform = transform


    def __getitem__(self, idx):
        img_path = self.pathes[idx]
        category = self.labels[idx]
        image = PImage.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        category = np.array(category)
        category = category.astype(np.int)
        category = torch.from_numpy(category)
        return image, category

    def __len__(self):
        return len(self.labels)

def score_predict(model, x):
    batch_size = x.size(0)
    score = model(x)
    score = F.softmax(score, dim=1)
    _,prediction = score.max(1)

    return prediction

def accuracy(iter, model):
    total = 0.0
    correct = 0.0

    with torch.no_grad():
        for images, labels in iter:
            images = Variable(images).cuda()
            preds = score_predict(model, images)
            total += labels.size(0)
            correct += (preds.cpu().data == labels).sum().item()
    print(correct)
    print(total)
    return correct/total


img_path = '/home/ubuntu/MURA/MURA-v1.1/train_image_paths.csv'
label_path = '/home/ubuntu/MURA/MURA-v1.1/train_labeled_studies.csv'
imgval_path = '/home/ubuntu/MURA/MURA-v1.1/valid_image_paths.csv'
labelval_path = '/home/ubuntu/MURA/MURA-v1.1/valid_labeled_studies.csv'
src = '/home/ubuntu/MURA/'

print(torch.cuda.is_available())

transform_image = transforms.Compose([
        transforms.Resize ([224, 224]),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

batchsize = 64
epouch = 90
learning_rate = 0.1
best_result = 0

MURA_dataset_train = CD_Dataset(src,img_path, label_path,transform=transform_image)
MURA_dataset_test = CD_Dataset(src,imgval_path, labelval_path,transform=transform_image)
MURA_trainloader = data.DataLoader(MURA_dataset_train, batch_size=batchsize)
MURA_testloader = data.DataLoader(MURA_dataset_test, batch_size=batchsize)

num_batch = len(MURA_trainloader)

resNet = resnet18()
resNet.cuda()

params = [
    {'params': resNet.parameters()}
]

optimizer = optim.SGD(params, lr=learning_rate, weight_decay=5e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
criterion = nn.CrossEntropyLoss()


for e in range(epouch):
    resNet.train()
    running_loss = 0.0
    for x_batch, y_batch in MURA_trainloader:
        
        #y_batch = torch.from_numpy(y_batch)
        x_batch, y_batch = Variable(x_batch.cuda()), Variable(y_batch.cuda(async=True))
        prediction = resNet(x_batch)
        
        loss = criterion(prediction, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_loss = loss.data[0]        
        running_loss += batch_loss


    resNet.eval()

    train_accuracy = accuracy(MURA_trainloader, resNet)
    test_accuracy = accuracy(MURA_testloader, resNet)
    
    resNet.train()
    if(test_accuracy>=best_result):
        best_result = test_accuracy

    print('Epoch [%d], Loss: %.4f, Train accuracy: %.4f, Test accuracy: %.4f, Best: %.4f' % (e, running_loss/num_batch, train_accuracy, test_accuracy, best_result))


True


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:125: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


8280.0
13457.0
661.0
1199.0
Epoch [0], Loss: 0.4653, Train accuracy: 0.6153, Test accuracy: 0.5513, Best: 0.5513
8280.0
13457.0
661.0
1199.0
Epoch [1], Loss: 0.3793, Train accuracy: 0.6153, Test accuracy: 0.5513, Best: 0.5513
8280.0
13457.0
661.0
1199.0
Epoch [2], Loss: 0.3529, Train accuracy: 0.6153, Test accuracy: 0.5513, Best: 0.5513
8280.0
13457.0
661.0
1199.0
Epoch [3], Loss: 0.4663, Train accuracy: 0.6153, Test accuracy: 0.5513, Best: 0.5513
8280.0
13457.0
661.0
1199.0
Epoch [4], Loss: 0.5352, Train accuracy: 0.6153, Test accuracy: 0.5513, Best: 0.5513
8280.0
13457.0
661.0
1199.0
Epoch [5], Loss: 0.5554, Train accuracy: 0.6153, Test accuracy: 0.5513, Best: 0.5513
8280.0
13457.0
661.0
1199.0
Epoch [6], Loss: 0.4997, Train accuracy: 0.6153, Test accuracy: 0.5513, Best: 0.5513
8280.0
13457.0
661.0
1199.0
Epoch [7], Loss: 0.5785, Train accuracy: 0.6153, Test accuracy: 0.5513, Best: 0.5513
8280.0
13457.0
661.0
1199.0
Epoch [8], Loss: 0.6156, Train accuracy: 0.6153, Test accuracy: 0.55